In [ ]:
import pandas as pd
import geopandas as gpd
import xarray as xr
import numpy as np
import datetime
import rasterio

In [2]:
import sys
sys.path.append('..')

In [3]:
import rsutils.modify_images

In [4]:
chirps_catalog_df = pd.read_csv('/gpfs/data1/cmongp2/sasirajann/nh_crop_calendar/crop_calendar/data/CHIRPS-v3.0/catalog.csv').sort_values(by='date')

In [ ]:
chirps_catalog_df.head()

In [6]:
region_gdf = gpd.read_file('/gpfs/data1/cmongp1/sasirajann/togo/shapefiles/Shapefiles/tgo_admbnda_adm0_inseed_itos_20210107.shp')

In [ ]:
# data_meta_list = \
# rsutils.modify_images.load_images(
#     src_filepaths = chirps_catalog_df['local_filepath'],
#     shapes_gdf = region_gdf,
#     njobs = 120,
# )

# datacube = np.concatenate([data_meta[0] for data_meta in data_meta_list])
# meta = data_meta_list[0][1]
# valid_time = chirps_catalog_df['date'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d')).to_numpy()

In [8]:
# np.save(
#     '/gpfs/data1/cmongp2/sasirajann/nh_crop_calendar/crop_calendar/data/togo_datacube/datacube.npy',
#     np.concatenate([data_meta[0] for data_meta in data_meta_list])
# )

# np.save(
#     '/gpfs/data1/cmongp2/sasirajann/nh_crop_calendar/crop_calendar/data/togo_datacube/metadata.pickle.npy',
#     {
#         'geotiff_metadata': data_meta_list[0][1],
#         'valid_time': chirps_catalog_df['date'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d')).to_numpy()
#     },
#     allow_pickle = True,
# )

In [9]:
datacube = np.load('/gpfs/data1/cmongp2/sasirajann/nh_crop_calendar/crop_calendar/data/togo_datacube/datacube.npy')
metadata = np.load('/gpfs/data1/cmongp2/sasirajann/nh_crop_calendar/crop_calendar/data/togo_datacube/metadata.pickle.npy', allow_pickle=True)[()]

valid_time = metadata['valid_time']
meta = metadata['geotiff_metadata']

In [10]:
chirps_v3_togo_da = xr.DataArray(
    datacube,
    coords = {
        'valid_time': valid_time,
    },
    dims = ('valid_time', 'x', 'y'),
)

In [ ]:
chirps_v3_togo_da

In [12]:
window = 10

rev = chirps_v3_togo_da.sel(valid_time=chirps_v3_togo_da.valid_time[::-1])

# rolling on reversed series (NaNs will be on the reversed "start", i.e. original end)
rolling_rev = rev.rolling(valid_time=window, min_periods=window).sum()

# reverse back to original time order
rolling = rolling_rev.sel(valid_time=rolling_rev.valid_time[::-1])

# rolling_doy_mean = rolling.groupby('valid_time.dayofyear').mean('valid_time').sel(dayofyear=slice(1, 365))

In [13]:
start_year = 1984
end_year = 2024

rolling_mean_1984_2024 = rolling.where(
    (rolling.valid_time.dt.year >= start_year) &
    (rolling.valid_time.dt.year < end_year) &
    (rolling.valid_time.dt.dayofyear >= 1) &
    (rolling.valid_time.dt.dayofyear <= 365),
    drop = True,
).groupby('valid_time.dayofyear').mean('valid_time').sel(dayofyear=slice(1, 365))

In [14]:
start_year = 1984
end_year = 2004

rolling_mean_1984_2004 = rolling.where(
    (rolling.valid_time.dt.year >= start_year) &
    (rolling.valid_time.dt.year < end_year) &
    (rolling.valid_time.dt.dayofyear >= 1) &
    (rolling.valid_time.dt.dayofyear <= 365),
    drop = True,
).groupby('valid_time.dayofyear').mean('valid_time').sel(dayofyear=slice(1, 365))

In [15]:
start_year = 2004
end_year = 2024

rolling_mean_2004_2024 = rolling.where(
    (rolling.valid_time.dt.year >= start_year) &
    (rolling.valid_time.dt.year < end_year) &
    (rolling.valid_time.dt.dayofyear >= 1) &
    (rolling.valid_time.dt.dayofyear <= 365),
    drop = True,
).groupby('valid_time.dayofyear').mean('valid_time').sel(dayofyear=slice(1, 365))

In [ ]:
rolling

In [ ]:
year = 2024

rolling.where(
    rolling.valid_time.dt.year.isin(year) &
    (rolling.valid_time.dt.dayofyear >= 1) &
    (rolling.valid_time.dt.dayofyear <= 365),
    drop = True,
).sel(x = 50, y=20).plot()

In [ ]:
rolling_mean_1984_2004.sel(x = 50, y=20).plot()

In [ ]:
rolling_mean_2004_2024.sel(x = 50, y=20).plot()

In [20]:
import sklearn.cluster
import rsutils.utils
import pandas as pd
import os

In [21]:
def relabel_clusters_by_count(cluster_ids:np.ndarray):
    _ids, _counts = np.unique(cluster_ids, return_counts=True)
    cluster_count_df = pd.DataFrame(data={
        'cluster_id': _ids,
        'count': _counts
    })
    cluster_count_df = cluster_count_df.sort_values(by='count', ascending=False)
    cluster_count_df['new_cluster_id'] = range(_ids.shape[0])
    new_cluster_id_map = dict(zip(
        cluster_count_df['cluster_id'],
        cluster_count_df['new_cluster_id'],
    ))
    new_cluster_ids = np.zeros(shape=cluster_ids.shape)
    for old_id, new_id in new_cluster_id_map.items():
        new_cluster_ids[cluster_ids == old_id] = new_id
    return new_cluster_ids.astype(int)

In [22]:
export_folderpath = '/gpfs/data1/cmongp2/sasirajann/nh_crop_calendar/crop_calendar/data/togo'

In [23]:
# def plot_clustered_lineplots(...

import matplotlib.pyplot as plt
import seaborn as sns


def plot_clustered_lineplots(
    crop_name:str,
    band_name:str,
    timeseries:np.ndarray,
    x:list,
    cluster_ids:np.ndarray, 
    save_filepath:str,
    alpha:float=0.05,
    y_min:float=-0.3,
    y_max:float=0.9,
    scale:float=5,
    aspect_ratio:float=2,
    nrows:int=3,
    ncols:int=3,
    limit_plots_per_cluster:int=1000,
    random_state:int=42,
    x_label:str='dates',
    cluster_id_to_color_map:dict=None,
    x_label_rotation:float = 0,
    ignore_clusters = [],
):
    n_points, n_timestamps = timeseries.shape

    if len(x) != n_timestamps:
        raise ValueError('Length of y should match timeseries shape.')
    
    unique_cluster_ids, counts = np.unique(cluster_ids, return_counts=True)
    cluster_counts_df = pd.DataFrame(data={'cluster_id':unique_cluster_ids,'count':counts})
    sorted_cluster_ids = cluster_counts_df.sort_values(
        by=['count', 'cluster_id'], ascending=[False, True],
    )['cluster_id']

    n_clusters = len(unique_cluster_ids)
    if n_clusters - len(set(ignore_clusters)) > nrows * ncols:
        raise ValueError(
            f'Too many clusters for too less nrows and ncols. '
            f'n_clusters = {n_clusters} > nrows * ncols = {nrows * ncols}'
        )

    _df = pd.DataFrame(
        data=np.concatenate([
            np.array([range(n_points)]).T, 
            timeseries, 
            np.array([cluster_ids]).T
        ], axis=1),
        columns=['id'] + list(x) + ['cluster_id']
    )
    _df['id'] = _df['id'].astype(int)
    _df['cluster_id'] = _df['cluster_id'].astype(int)
    melted_dfs = []
    for cluster_id in sorted_cluster_ids:
        df_to_melt_i = _df[_df['cluster_id']==cluster_id]
        melted_df_i = df_to_melt_i[_df.columns[:-1]].melt(
            id_vars='id',
            var_name=x_label, 
            value_name=band_name,
        )
        melted_df_i['cluster_id'] = cluster_id
        melted_dfs.append(melted_df_i)
    _df_melted = pd.concat(melted_dfs)

    fig, axs = plt.subplots(nrows=nrows, ncols=ncols, figsize=(scale*aspect_ratio*ncols, scale*nrows))
    i, j = 0, 0

    for index, cluster_number in enumerate(sorted_cluster_ids):
        if cluster_number in ignore_clusters:
            continue
        
        if ncols > 1 and nrows > 1:
            ax = axs[j][i]
            if i == ncols:
                j += 1
                i = 0
        else:
            ax = axs[i]
        
        i += 1
            
        _df_plot = _df_melted[_df_melted['cluster_id']==cluster_number]
        count = _df_plot['id'].unique().shape[0]
        ids = _df_plot['id'].unique().tolist()
        np.random.RandomState(seed=random_state).shuffle(ids)
        selected_ids = ids[:limit_plots_per_cluster]
        _df_plot = _df_plot[_df_plot['id'].isin(selected_ids)]
        limited_count = _df_plot['id'].unique().shape[0]

        if cluster_id_to_color_map is None:
            color = sns.palettes.color_palette()[index % 10]
        else:
            color = cluster_id_to_color_map[cluster_number]

        g = sns.lineplot(
            data=_df_plot,
            ax=ax,
            x=x_label,
            y=band_name,
            hue='id',
            alpha=alpha,
            legend=False,
            palette=[
                color for _ in range(limited_count)
            ],
        )
        g.set_ylim([y_min, y_max])
        g.set_title(f'crop: {crop_name}, cluster_id: {cluster_number}, count: {count}')
        g.grid()

        if x_label_rotation != 0:
            g.set_xticklabels(g.get_xticklabels(), rotation=x_label_rotation)

    if nrows > 1 and ncols > 1:
        while j < nrows:
            fig.delaxes(axs[j][i])
            i += 1
            if i == ncols:
                j += 1
                i = 0

    fig.savefig(save_filepath, bbox_inches='tight')

    plt.close()
    plt.cla()
    plt.clf()


In [ ]:
filename = f'rolling{window}_1984-2024_doy_mean'
band_name = f'rolling sum ({window} days) rainfall DOY mean'
data = rolling_mean_1984_2024

n_ts, height, width = data.values.shape

data_2d = data.values.reshape(n_ts, height*width).swapaxes(0, 1)
data_2d.shape

y_max = np.ceil(data.max().data)

n_clusters = 3
nrows, ncols = 2, 1

cluster_ids = sklearn.cluster.MiniBatchKMeans(
    n_clusters = n_clusters,
    random_state = 42,
).fit(data_2d).labels_

cluster_ids = relabel_clusters_by_count(cluster_ids=cluster_ids)

plot_clustered_lineplots(
    crop_name = '',
    band_name = band_name,
    timeseries = data_2d,
    cluster_ids = cluster_ids,
    y_min = -1,
    y_max = y_max,
    nrows = nrows,
    ncols = ncols,
    x = range(1, n_ts + 1),
    x_label = 'DOY',
    save_filepath = os.path.join(export_folderpath, f'{filename}.png'),
    alpha = 0.02,
    aspect_ratio = 1.5,
    ignore_clusters = [0],
)

In [25]:
cluster_ids_2d = cluster_ids.reshape(height, width).astype(np.uint8)

In [26]:
meta.update({
    'dtype': 'uint8',
    'nodata': 0,
})

with rasterio.open(os.path.join(export_folderpath, f'{filename}.tif'), 'w', **meta) as dst:
    dst.write(cluster_ids_2d, 1)

## Bimodality tests

In [27]:
import scipy.signal

In [28]:
not_bimodal = rolling_mean_1984_2004.sel(x = 20, y=20).data

In [29]:
bimodal = rolling_mean_2004_2024.sel(x = 90, y=20).data

In [30]:
shouldnt_by_bimodal = rolling_mean_1984_2004.sel(x = 80, y=20).data

In [31]:
def get_prominence_ratio(timeseries,  prominence=10):
    peaks, props = scipy.signal.find_peaks(timeseries, prominence=prominence)
    n_peaks = len(peaks)

    if n_peaks >= 2:
        prom = props["prominences"]
        prom_ratio = prom[np.argsort(not_bimodal[peaks])][-2:]
        prominence_ratio = min(prom_ratio) / max(prom_ratio)
    else:
        prominence_ratio = 0
    
    return prominence_ratio

In [66]:
def get_harmonic_ratio(timeseries):
    fft = np.fft.rfft(timeseries - timeseries.mean())
    amps = np.abs(fft)
    A1 = amps[1]   # 1 cycle/yr
    A2 = amps[2]   # 2 cycles/yr
    BPI = A2 / (A1 + 1e-12)
    return BPI

In [ ]:
scale = 5
aspect_ratio = 1.3
fig, ax = plt.subplots(figsize=(scale*aspect_ratio, scale))

for ts, label in [
    (not_bimodal, 'not bimodal'),
    (bimodal, 'bimodal'),
    # (shouldnt_by_bimodal, 'should not be bimodal'),
]:
    pr = get_prominence_ratio(ts)
    hr = get_harmonic_ratio(ts)
    ax.plot(ts, label=f'{label} (PR = {round(pr, 2)}, HR = {round(hr, 2)})')

ax.legend()

In [33]:
# test randomly selected cluster, or plot distrbution on the cluster

In [ ]:
rolling_mean_1984_2024.shape

In [58]:
data = rolling_mean_1984_2024

_, nx, ny = data.shape

pr_array = np.zeros(shape=(nx, ny))
hr_array = np.zeros(shape=(nx, ny))

for x in range(nx):
    for y in range(ny):
        pr = get_prominence_ratio(
            timeseries = data.sel(x=x, y=y).data,
            prominence = 10,
        )
        hr = get_harmonic_ratio(
            timeseries = data.sel(x=x, y=y).data,
        )
        pr_array[x, y] = pr
        hr_array[x, y] = hr

In [ ]:
cluster_pr_df = pd.DataFrame(
    data = {
        'cluster id': cluster_ids_2d.flatten(),
        'prominence ratio': pr_array.flatten(),
        'harmonic ratio': hr_array.flatten(),
    }
)

cluster_pr_df = cluster_pr_df[cluster_pr_df['cluster id'] != 0]

scale = 5
aspect_ratio = 1
ncols = 2
NBINS = 50

fig, axs = plt.subplots(figsize=(scale*ncols, scale*aspect_ratio), ncols=ncols)

g = sns.histplot(
    ax = axs[0],
    data = cluster_pr_df,
    x = 'prominence ratio',
    hue = 'cluster id',
    bins = NBINS,
)

g = sns.histplot(
    ax = axs[1],
    data = cluster_pr_df,
    x = 'harmonic ratio',
    hue = 'cluster id',
    bins = NBINS,
)

In [ ]:
scale = 4
aspect_ratio = 2
ncols = 2
fig, axs = plt.subplots(figsize=(scale*ncols, scale*aspect_ratio), ncols=ncols)

g = sns.heatmap(data=pr_array, ax=axs[0])
_ = g.set_title('Prominence Ratio Test')

g = sns.heatmap(data=hr_array, ax=axs[1])
_ = g.set_title('Harmonic Ratio Test')

In [54]:
ts = shouldnt_by_bimodal

In [ ]:
BPI

In [ ]:
1